# Equity Linked Notes - Autocallable Athena (basket)

This notebook demonstrates how to access and use the functionalities of **Autocallable Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Athena structure with basket underlying.

**You will be able to:**
* Define Athena instrument with basic parameters
* Configure pricing parameters
* Evaluate Athena Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Autocallable Athena, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Athena_basket",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Basket", type = "string", value="RACE_MI|MBGn_DE|BMWG_DE|VOWG_DE"),
        sp.NameTypeValue(name="FirstAsset", type = "string", value="Perf_RACE_MI"),
        sp.NameTypeValue(name="LastAsset", type = "string", value="Perf_VOWG_DE"),
        sp.NameTypeValue(name="BasketFunction", type = "string", value="Average"),
        sp.NameTypeValue(name="BasketPerf", type = "string", value="If(\"BasketFunction\" == \"WorstOf\", Min(FirstAsset[t]:LastAsset[t]), IF(\"BasketFunction\" == \"Average\", Average(FirstAsset[t]:LastAsset[t],True), 0))"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="10/07/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="24/07/2029"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="10/07/2029"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["10/10/2025", "10/10/2025", "24/10/2025", "24/10/2025"],
                    ["12/01/2026", "12/01/2026", "26/01/2026", "26/01/2026"],
                    ["10/04/2026", "10/04/2026", "24/04/2026", "24/04/2026"],
                    ["10/07/2026", "10/07/2026", "24/07/2026", "24/07/2026"],
                    ["12/10/2026", "12/10/2026", "26/10/2026", "26/10/2026"],
                    ["11/01/2027", "11/01/2027", "25/01/2027", "25/01/2027"],
                    ["12/04/2027", "12/04/2027", "26/04/2027", "26/04/2027"],
                    ["12/07/2027", "12/07/2027", "26/07/2027", "26/07/2027"],
                    ["11/10/2027", "11/10/2027", "25/10/2027", "25/10/2027"],
                    ["10/01/2028", "10/01/2028", "24/01/2028", "24/01/2028"],
                    ["10/04/2028", "10/04/2028", "24/04/2028", "24/04/2028"],
                    ["10/07/2028", "10/07/2028", "24/07/2028", "24/07/2028"],
                    ["10/10/2028", "10/10/2028", "24/10/2028", "24/10/2028"],
                    ["10/01/2029", "10/01/2029", "24/01/2029", "24/01/2029"],
                    ["10/04/2029", "10/04/2029", "24/04/2029", "24/04/2029"]]),
        sp.NameTypeValue(name="FinalCouponBarrier", type = "string", value="100%"),
        sp.NameTypeValue(name="AutocallBarrier", type = "curve", value=[["10/07/2025","100%"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="50%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="1.875%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%"),
        sp.NameTypeValue(name="VarInit", type = "string", value="0")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Repeat(Basket,#)",
            "Repeat(Basket,Perf_#)",
            "Performance",
            "Alive",
            "Early",
            "Count",
            "Coupon",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(#)",
            "",
            "",
            "1",
            "",
            "$n=VarInit;$n",
            "",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(#)",
            "#[LastDate]/#[StrikeDate]",
            "BasketPerf",
            "If(Performance[t] >= Interpol(AutocallBarrier,PS()), 0, Alive[LastDate])",
            "(1 - Alive[t]) * Alive[LastDate(-1)]",
            "$n=$n+1;$n",
            "Receive $n*CouponRate * Early[t]",
            "Receive Early[t]",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(#)",
            "#[LastDate]/#[StrikeDate]",
            "BasketPerf",
            "",
            "",
            "$n=$n+1;$n",
            "Receive (MaturityDate, If(Performance[t]>=FinalCouponBarrier, $n*CouponRate, 0) * Alive[LastDate])",
            "Receive (MaturityDate, Alive[LastDate])",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)*Alive[LastDate]))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

athena_basket = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

athena_basket_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 8, 25),  # Set your desired valuation date
    report_ccy="USD",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "RACE.MI",
                  underlying_name = "RACE_MI",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "MBGn.DE",
                  underlying_name = "MBGn_DE",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "BMWG.DE",
                  underlying_name = "BMWG_DE",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "VOWG.DE",
                  underlying_name = "VOWG_DE",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing

try:
    response = sp.price(
        definitions=[athena_basket],
        pricing_preferences=athena_basket_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )

    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Athena_basket",
    "dealCcy": "EUR",
    "reportCcy": "USD",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 1.595764405,
        "BMWG_DE": 1340.639992865,
        "Count": 128.85727191,
        "Coupon": 0.027003047,
        "Early": 0.92024578,
        "MBGn_DE": 791.56609168,
        "OptionAtMaturity": -0.008178851,
        "PERF_BMWG_DE": 15.698360572,
        "PERF_MBGn_DE": 14.904275874,
        "PERF_RACE_MI": 14.957031183,
        "PERF_VOWG_DE": 15.267764122,
        "Performance": 15.206857938,
        "Price": 1007.136036976,
        "PriceIn%": 100.713603698,
        "RACE_MI": 6352.251143611,
        "Settlement": 0.988311841,
        "VOWG_DE": 1486.316837248
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be 

#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_athena_basket_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_athena_basket_valuation)

,Field,Value
0,marketValueInDealCcy,1007.136037
1,marketValueInReportCcy,1170.141005


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=athena_basket_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Alive', 'Early', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,RACE_MI,MBGn_DE,BMWG_DE,VOWG_DE,PERF_RACE_MI,PERF_MBGn_DE,PERF_BMWG_DE,PERF_VOWG_DE,Performance,Alive,Early,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-07-10,1.0000,424.700000,53.110000,85.400000,97.350000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0000,0.0000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2025-10-10,0.9974,403.446551,54.574196,91.163005,104.105742,0.949957,1.027569,1.067482,1.069396,1.028601,0.2668,0.7332,0.013737,0.732626,0.000000,0.000000,EUR,Future
2,2026-01-12,0.9923,406.736406,54.817425,91.596151,104.625525,0.957703,1.032149,1.072554,1.074736,1.034285,0.1686,0.0982,0.003680,0.098127,0.000000,0.000000,EUR,Future
3,2026-04-10,0.9877,407.116187,55.035023,92.093417,104.832714,0.958597,1.036246,1.078377,1.076864,1.037521,0.1282,0.0404,0.002271,0.040371,0.000000,0.000000,EUR,Future
4,2026-07-10,0.9831,406.548424,52.708664,88.329647,100.129438,0.957260,0.992443,1.034305,1.028551,1.003140,0.1156,0.0126,0.000944,0.012591,0.000000,0.000000,EUR,Future
5,2026-10-12,0.9783,408.331805,52.972689,89.107672,100.714914,0.961459,0.997415,1.043415,1.034565,1.009214,0.1068,0.0088,0.000824,0.008794,0.000000,0.000000,EUR,Future
6,2027-01-11,0.9737,410.377969,53.299847,89.576298,101.454131,0.966277,1.003575,1.048903,1.042159,1.015228,0.1002,0.0066,0.000742,0.006595,0.000000,0.000000,EUR,Future
7,2027-04-12,0.9689,412.423134,53.586546,89.955469,101.722336,0.971093,1.008973,1.053343,1.044914,1.019580,0.0940,0.0062,0.000813,0.006195,0.000000,0.000000,EUR,Future
8,2027-07-12,0.9640,411.448738,50.818426,85.949258,95.266906,0.968799,0.956852,1.006432,0.978602,0.977671,0.0920,0.0020,0.000300,0.001998,0.000000,0.000000,EUR,Future
9,2027-10-11,0.9589,414.043789,51.225508,86.772125,95.630574,0.974909,0.964517,1.016067,0.982338,0.984458,0.0888,0.0032,0.000540,0.003197,0.000000,0.000000,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_athena_basket_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_athena_basket_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,2.549520
1,deltaAmountInReportCcy,2.962160
2,gammaAmountInDealCcy,-0.814968
3,gammaAmountInReportCcy,-0.946871
4,vegaAmountInDealCcy,-1.360110
5,vegaAmountInReportCcy,-1.580244
6,thetaAmountInDealCcy,-0.329483
7,thetaAmountInReportCcy,-0.382810
